In [29]:
import pandas as pd
import re
from functools import reduce
from glob import glob

In [30]:
def TDLog(file):
    List1=[]
    with open(file, 'r') as f:
        pat = '^\"(.+?)\" #[0-9]+.*nid=.+? (.+?) \[.*$'
        r = re.compile(pat)
        lines = f.readlines()
        line1=None
        for data in lines:
            if("tid=0x" in data):
                #print(data)
                line1 = re.search(pat,data)
                if line1 is not None:
                    List1.append([line1.group(1),line1.group(2)])
    columns=['thread','state']
    print(f"{file.split('/')[-1]} Total Threads = {len(List1)}")
    df = pd.DataFrame(List1)
    df.columns=columns
    df['tn'] = df['thread'].str.split('-',expand=True)[0]
    return df

def genColNames(length):
    collist = ['thread','state']
    for i in range(1,length+1):
        collist.append(f"td{i}")
    return collist


def merged(dflist):
    data_frames=[]
    columns = genColNames(len(dflist))
    for df in dflist:
        data_frames.append(df.groupby(['tn','state']).count().sort_values(('thread'),ascending=False).reset_index())
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tn','state'],
                                                how='outer'), data_frames)
    df_merged.columns=columns
    return df_merged

In [31]:
directory = 'tmp/*'
LIST = []
for file in glob(directory):
    LIST.append(TDLog(file))
merged(LIST).sort_values(['thread','td1'],ascending=[True,False])

td1 Total Threads = 60
td3 Total Threads = 60
td2 Total Threads = 60


,thread,state,td1,td2,td3
13,AWT,in Object.wait(),1,1,1
26,AWT,runnable,1,1,1
28,AWT,waiting on condition,1,1,1
29,AWTThreading pool,waiting on condition,1,1,1
30,ApplicationImpl pooled thread 138,waiting on condition,1,1,1
31,ApplicationImpl pooled thread 140,waiting on condition,1,1,1
32,ApplicationImpl pooled thread 141,waiting on condition,1,1,1
33,ApplicationImpl pooled thread 142,waiting on condition,1,1,1
34,ApplicationImpl pooled thread 143,waiting on condition,1,1,1
35,ApplicationImpl pooled thread 144,waiting on condition,1,1,1
